In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/the-marvel-universe-social-network/nodes.csv
/kaggle/input/the-marvel-universe-social-network/edges.csv
/kaggle/input/the-marvel-universe-social-network/hero-network.csv


In [2]:
import pandas as pd 
import networkx as nx

from itertools import combinations

In [3]:
node_df= pd.read_csv("/kaggle/input/the-marvel-universe-social-network/nodes.csv")
edge_df= pd.read_csv("/kaggle/input/the-marvel-universe-social-network/edges.csv")
hero_net_df= pd.read_csv("/kaggle/input/the-marvel-universe-social-network/hero-network.csv")

# 1. Dataset Exploration & Preprocessing
Before further analysis let's explore and let's do some required processing

In [4]:
node_df.head()

,node,type
0,2001 10,comic
1,2001 8,comic
2,2001 9,comic
3,24-HOUR MAN/EMMANUEL,hero
4,3-D MAN/CHARLES CHAN,hero


In [5]:
edge_df.head()

,hero,comic
0,24-HOUR MAN/EMMANUEL,AA2 35
1,3-D MAN/CHARLES CHAN,AVF 4
2,3-D MAN/CHARLES CHAN,AVF 5
3,3-D MAN/CHARLES CHAN,COC 1
4,3-D MAN/CHARLES CHAN,H2 251


In [6]:
hero_net_df.head()

,hero1,hero2
0,"LITTLE, ABNER",PRINCESS ZANDA
1,"LITTLE, ABNER",BLACK PANTHER/T'CHAL
2,BLACK PANTHER/T'CHAL,PRINCESS ZANDA
3,"LITTLE, ABNER",PRINCESS ZANDA
4,"LITTLE, ABNER",BLACK PANTHER/T'CHAL


## 1.1 Extraction of Hero Name
based on "/", it looks like the name of the hero on the left and the name of the person on the right is not the hero. We need to split the column. But at first we need to check what is length of hero name. So Let's split accordingly

In [7]:
for c in ['hero1','hero2']:
    print(f"Maximum length of {c} column in hero_net_df.csv:{max(hero_net_df[c].apply(lambda x: len(x)))}")

Maximum length of hero1 column in hero_net_df.csv:20
Maximum length of hero2 column in hero_net_df.csv:20


In [8]:
for c in ['hero1','hero2']:
    hero_net_df[c]= hero_net_df[c].apply(lambda x: x[:20].split("/")[0])

edge_df['hero']= edge_df['hero'].apply(lambda x:x[:20].split("/")[0])

## 1.2 Dataset Reconstruction

In [9]:
top30_hero= edge_df.groupby(['hero'])[['comic']].count().sort_values(by=['comic'], ascending=False).head(30).index
top30_hero

Index(['SPIDER-MAN', 'CAPTAIN AMERICA', 'IRON MAN', 'THING', 'THOR',
       'HUMAN TORCH', 'MR. FANTASTIC', 'HULK', 'WOLVERINE', 'INVISIBLE WOMAN',
       'SCARLET WITCH', 'BEAST', 'DR. STRANGE', 'WATSON-PARKER, MARY',
       'DAREDEVIL', 'HAWK', 'VISION', 'CYCLOPS', 'WASP', 'JAMESON, J. JONAH',
       'ANT-MAN', 'SUB-MARINER', 'STORM', 'PROFESSOR X', 'FURY, COL. NICHOLAS',
       'MARVEL GIRL', 'COLOSSUS II', 'ANGEL', 'NIGHTCRAWLER', 'ICEMAN'],
      dtype='object', name='hero')

In [10]:
temp1 = set(edge_df[edge_df['hero']=='SPIDER-MAN']['comic'])
temp2 = set(edge_df[edge_df['hero']=='HULK']['comic'])
hero_1 = []; hero_2 = []; count = [];
for comb in list(combinations(top30_hero, 2)):    
    temp1 = set(edge_df[edge_df['hero']==comb[0]]['comic'])
    temp2 = set(edge_df[edge_df['hero']==comb[1]]['comic'])
    cnt = len(temp1.intersection(temp2)) # Appear Together    
    hero_1.append(comb[0]); hero_2.append(comb[1]); count.append(cnt);
df = pd.DataFrame({'Hero_1':hero_1, 'Hero_2':hero_2, 'Count':count})
display(df.head())

,Hero_1,Hero_2,Count
0,SPIDER-MAN,CAPTAIN AMERICA,145
1,SPIDER-MAN,IRON MAN,95
2,SPIDER-MAN,THING,125
3,SPIDER-MAN,THOR,96
4,SPIDER-MAN,HUMAN TORCH,147
